In [1]:
import sys
sys.path.append('../')

from glob import glob
import os
from tqdm import tqdm
import numpy as np

from EM import NiftiManager, Evaluate, EM, ElastixTransformix, FileManager, BrainAtlasManager
from utils.fusion import majority_voting_fusion, weighted_voting_fusion

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [2]:
NM    = NiftiManager()
EVAL  = Evaluate()
ET    = ElastixTransformix()
FM    = FileManager()
BM    = BrainAtlasManager()

In [3]:
print(os.listdir('./output/'))
print(os.listdir('./output/images'))
print(os.listdir('./output/labels'))

['images', 'labels']
['output_IBSR_11', 'output_IBSR_12', 'output_IBSR_13', 'output_IBSR_14', 'output_IBSR_17']
['output_IBSR_11', 'output_IBSR_12', 'output_IBSR_13', 'output_IBSR_14', 'output_IBSR_17']


In the next few cells, we will run the fusion and evaluation using registered intensities and labelled that were registered to all val/test subjects, not taking into account the group or any other information that were analyzed in the EDA notebook.

In [ ]:
# {fixed}/{moving_folders}
# each fixed will be the test subject (for now, it is the validation), thus we will validate separately

output_images_dir = './output/images'
output_labels_dir = './output/labels'

# iterate through all fusion techniques
for fusion_callback in [majority_voting_fusion, weighted_voting_fusion]:
    print(f'Evaluating using "{fusion_callback.__name__}" fusion technique')

    # iterate through all fixed directories
    for fixed_intensity, fixed_label in zip(os.listdir(output_images_dir), os.listdir(output_labels_dir)):
        # create a path for the current fixed
        fixed_label_path = os.path.join(output_labels_dir, fixed_label)
        fixed_intensity_path = os.path.join(output_labels_dir, fixed_intensity)
        
        # get the propagated masks that were result of the label propagation to the current fixed
        moving_masks = [m.replace('\\', '/') for m in glob(os.path.join(fixed_label_path, "IBSR_**_seg", "*.nii"), recursive=True)]
        moving_intensity = [m.replace('\\', '/') for m in glob(os.path.join(fixed_intensity_path, "IBSR_**", "*.nii"), recursive=True)]
        
        # load the label gt of the test image (target) ( validation for now )
        fixed_filename = fixed_intensity.split('output_')[-1] # output_IBSR_11 > IBSR_11
        target_label_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}_seg.nii.gz'
        target_intensity_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}.nii.gz'
    
        target_label_nifti = NM.load_nifti(target_label_path)[0][:,:,:,0]
        target_intensity_nifti = NM.load_nifti(target_intensity_path)[0][:,:,:,0]
    
        # fuse method
        if fusion_callback.__name__ == 'majority_voting_fusion':
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                load_nifti_callback = NM.load_nifti)
            
        elif fusion_callback.__name__ == 'weighted_voting_fusion':
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                intensity_dirs = moving_intensity,
                target_intensity_path = target_intensity_path,
                load_nifti_callback = NM.load_nifti)
        
        # segment using the tissue atlases we created using the voting technique
        segmentation_result, concatenated_atlas = BM.segment_using_tissue_atlas(target_intensity_nifti, target_label_nifti, mean_csf, mean_gm, mean_wm)
    
        # evaluate dice score
        dice_coefficients = EVAL.evaluate_dice_volumes(segmentation_result, target_label_nifti, labels={'BG':0, 'CSF':1, 'GM':2, 'WM':3})
    
        print(f'Test volume {fixed_filename} dice results {dice_coefficients}')
    
    print('-----------------------------------------------------------------------------------')

Evaluating using "majority_voting_fusion" fusion technique
Test volume IBSR_11 dice results {'WM': 0.78578, 'GM': 0.876239, 'CSF': 0.740233}
Test volume IBSR_12 dice results {'WM': 0.800773, 'GM': 0.900176, 'CSF': 0.746292}
Test volume IBSR_13 dice results {'WM': 0.787228, 'GM': 0.90687, 'CSF': 0.585436}
Test volume IBSR_14 dice results {'WM': 0.805461, 'GM': 0.902183, 'CSF': 0.793226}
Test volume IBSR_17 dice results {'WM': 0.780982, 'GM': 0.900373, 'CSF': 0.83405}
-----------------------------------------------------------------------------------
Evaluating using "weighted_voting_fusion" fusion technique
Test volume IBSR_11 dice results {'WM': 0.792716, 'GM': 0.875556, 'CSF': 0.747126}
Test volume IBSR_12 dice results {'WM': 0.800848, 'GM': 0.895985, 'CSF': 0.724778}


code below is just for debugging, don't delete

In [14]:
labels_path_output_IBSR_11 = './output/labels/output_IBSR_11/'
images_path_output_IBSR_11 = './output/images/output_IBSR_11/'

labels_output_IBSR_11 = glob(os.path.join(labels_path_output_IBSR_11, "IBSR_**_seg", "*.nii"), recursive=True)
images_output_IBSR_11 = glob(os.path.join(images_path_output_IBSR_11, "IBSR_**", "*.nii"), recursive=True)

In [16]:
# images_output_IBSR_11

In [11]:
# # segmenting using those atlases
# IBSR_11 = '../TrainingValidationTestSets/Validation_Set/IBSR_11/IBSR_11.nii.gz'
# IBSR_11_label = '../TrainingValidationTestSets/Validation_Set/IBSR_11/IBSR_11_seg.nii.gz'

In [12]:
# mean_csf, mean_gm, mean_wm = weighted_voting(
#     labels_dirs = labels_output_IBSR_11, 
#     intensity_dirs = images_output_IBSR_11,
#     target_intensity_path = IBSR_11,
#     load_nifti_callback = NM.load_nifti)

# print(mean_csf.shape, mean_gm.shape, mean_wm.shape)


In [13]:
# NM.show_mean_volumes(mean_csf, mean_wm, mean_gm, slices=[128])